In [12]:
from langchain_community.graphs import Neo4jGraph
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI, OpenAI

import os
import textwrap

In [13]:
# Load from environment
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE') or 'neo4j'

In [14]:
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD
)

In [15]:
# Check the graph schema
kg.refresh_schema()
print(textwrap.fill(kg.schema, 60))

Node properties are the following: Movie {id: STRING,
duration: STRING, listed_in: STRING, date_added: STRING,
release_year: STRING, rating: STRING, description: STRING,
title: STRING},Person {name: STRING},Type {type:
STRING},Country {name: STRING},Year {value:
INTEGER},Category {name: STRING} Relationship properties are
the following:  The relationships are the following: (:Movie
)-[:TYPED_AS]->(:Type),(:Movie)-[:WHERE]->(:Country),(:Movie
)-[:CREATED_ON]->(:Year),(:Movie)-[:IN_CATEGORY]->(:Category
),(:Person)-[:ACTED_IN]->(:Movie),(:Person)-[:WORK_WITH]->(:
Person),(:Person)-[:DIRECTED]->(:Movie),(:Year)-[:NEXT]->(:Y
ear)


In [124]:
CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher statements for particular questions:

# How many people starred in Top Gun?
MATCH (m:Movie {{title:"Top Gun"}})<-[:ACTED_IN]-()
RETURN count(*) AS numberOfActors

# What are the most popular movie categories
MATCH (m:Movie)-[:IN_CATEGORY]->(c:Category)
WITH c.name AS Category, COUNT(m) AS MoviesCount
RETURN Category, MoviesCount
ORDER BY MoviesCount DESC

# All the movies Keanu Reeves acted in
MATCH (p:Person {{name: "Keanu Reeves"}})-[:ACTED_IN]->(m:Movie)
RETURN m.title AS Movie, m.release_year AS ReleaseYear
ORDER BY m.release_year DESC

# all persons who have acted in at least one movie with Keanu Reeves
MATCH (p:Person {{name: "Keanu Reeves"}})-[:ACTED_IN]->(m:Movie)<-[:ACTED_IN]-(coActors)
WHERE p.name <> coActors.name
RETURN DISTINCT coActors.name AS CoActor

# Find pairs of actors who have worked together in more than one movie
MATCH (p1:Person)-[:ACTED_IN]->(m:Movie)<-[:ACTED_IN]-(p2:Person)
WHERE p1.name < p2.name
WITH p1, p2, COUNT(m) AS sharedMovies
WHERE sharedMovies > 1
RETURN p1.name AS Actor1, p2.name AS Actor2, sharedMovies
ORDER BY sharedMovies DESC

# Retrieve movies along with their categories.
MATCH (m:Movie)-[:IN_CATEGORY]->(c:Category)
RETURN m.title AS Movie, collect(c.name) AS Categories

# Find Movies Directed by Actors
MATCH (p:Person)-[:DIRECTED]->(m:Movie), (p)-[:ACTED_IN]->(m2:Movie)
RETURN p.name AS Person, m.title AS DirectedMovie, m2.title AS ActedInMovie

# Find movies released in a United States grouped by their release year
MATCH (m:Movie)-[:WHERE]->(c:Country {{name: "United States"}})
RETURN m.release_year AS ReleaseYear, collect(m.title) AS Movies
ORDER BY m.release_year

# Which actors have worked with Keanu Reeves in more than 2 movies and list the movies
MATCH (p1:Person)-[:ACTED_IN]->(m:Movie)<-[:ACTED_IN]-(p2:Person)
WHERE p1.name = "Keanu Reeves" AND p1.name <> p2.name
WITH p1, p2, collect(m.title) AS Movies, COUNT(m) AS sharedMovies
WHERE sharedMovies > 2
RETURN p2.name AS Actor, Movies
ORDER BY sharedMovies DESC

The question is:
{question}"""

In [125]:
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], 
    template=CYPHER_GENERATION_TEMPLATE
)

In [126]:
cypherChain = GraphCypherQAChain.from_llm(
    OpenAI(temperature=0),
    graph=kg,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
)

In [127]:
cypherChain.invoke("Which actors have worked with Keanu Reeves in more than 2 movies and list the movies")



> Entering new GraphCypherQAChain chain...
Generated Cypher:


MATCH (p1:Person)-[:ACTED_IN]->(m:Movie)<-[:ACTED_IN]-(p2:Person)
WHERE p1.name = "Keanu Reeves" AND p1.name <> p2.name
WITH p1, p2, collect(m.title) AS Movies, COUNT(m) AS sharedMovies
WHERE sharedMovies > 2
RETURN p2.name AS Actor, Movies
ORDER BY sharedMovies DESC
Full Context:
[{'Actor': 'Laurence Fishburne', 'Movies': ['The Matrix', 'The Matrix Reloaded', 'The Matrix Revolutions']}, {'Actor': 'Hugo Weaving', 'Movies': ['The Matrix', 'The Matrix Reloaded', 'The Matrix Revolutions']}, {'Actor': 'Carrie-Anne Moss', 'Movies': ['The Matrix', 'The Matrix Reloaded', 'The Matrix Revolutions']}]

> Finished chain.


{'query': 'Which actors have worked with Keanu Reeves in more than 2 movies and list the movies',
 'result': '\nLaurence Fishburne has worked with Keanu Reeves in The Matrix, The Matrix Reloaded, and The Matrix Revolutions. Hugo Weaving has also worked with Keanu Reeves in The Matrix, The Matrix Reloaded, and The Matrix Revolutions. Carrie-Anne Moss has also worked with Keanu Reeves in The Matrix, The Matrix Reloaded, and The Matrix Revolutions.'}